In [1]:
# -*- coding: utf-8 -*-
"""
@author: ogouvert
"""

#%% 
import sys
sys.path.append("../../model/OrdNMF")
sys.path.append("../../function")

import os
import pickle as pickle

from OrdNMF import OrdNMF

import preprocess_data  as prep

#%% 
prop_test = 0.2
seed_test = 1001

with open('../../data/ML/ml_145_U1.29e+02_I4.72e+02_min_uc20_sc20', 'rb') as f:
    out = pickle.load(f)
    Y = out['Y_listen']
U,I = Y.shape

#%% Directory
Y_train,Y_test = prep.divide_train_test(Y,prop_test=prop_test,seed=seed_test)
save_dir = 'out/seed_%d' %(seed_test)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
        
#%% IN 
opt_hyper = ['beta']  

Ks = [25,50,100,150,200,250]
Seeds = range(5) # Seed of the different initializations
tol = 10**(-5)
min_iter = 0
max_iter = 10**5

#%% Ord NMF
alpha = .3 

# Ordinal
if True:
    for approx in [False]:
        for K in [150]:
            for seed in Seeds:
                model = OrdNMF(K=K, alphaW=alpha, alphaH=alpha)
                model.fit(Y_train, T=Y_train.max(), 
                          seed=seed, opt_hyper=opt_hyper, 
                          approx = approx, 
                          precision=tol, min_iter=min_iter, max_iter=max_iter,
                          save=True, save_dir=save_dir,prefix='ML', 
                          verbose=True)
   
# Binary  
if True:
    for approx in [True,False]: # Approx Bernoulli -> Poisson
        for K in [50, 100]:
            for seed in Seeds:
                R1 = Y_train>=1
                R1.eliminate_zeros()
                model = OrdNMF(K=K, alphaW=alpha, alphaH=alpha)
                model.fit(R1, T=1, 
                          seed=seed, opt_hyper=opt_hyper, 
                          approx = approx, 
                          precision=tol, min_iter=min_iter, max_iter=max_iter,
                          save=True, save_dir=save_dir,prefix='ML_geq1', 
                          verbose=False)
        
if True:
    for approx in [True,False]: # Approx Bernoulli -> Poisson
        for K in [50, 150]:
            for seed in Seeds:                
                R8 = Y_train>=8
                R8.eliminate_zeros()
                model = OrdNMF(K=K, alphaW=alpha, alphaH=alpha)
                model.fit(R8, T=1, 
                          seed=seed, opt_hyper=opt_hyper, 
                          approx = approx, 
                          precision=tol, min_iter=min_iter, max_iter=max_iter,
                          save=True, save_dir=save_dir,prefix='ML_geq8', 
                          verbose=False)

ITERATION #0
	Updates: time=0.30
	Rate:inf
ITERATION #1
	Updates: time=0.25
	Rate:0.7254286374284904
ITERATION #2
	Updates: time=0.25
	Rate:0.26062977737428844
ITERATION #3
	Updates: time=0.22
	Rate:0.03922236996801362
ITERATION #4
	Updates: time=0.18
	Rate:0.03283100513655056
ITERATION #5
	Updates: time=0.06
	Rate:0.037515186907046055
ITERATION #6
	Updates: time=0.06
	Rate:0.03803779177285054
ITERATION #7
	Updates: time=0.06
	Rate:0.03375879471239245
ITERATION #8
	Updates: time=0.06
	Rate:0.028103853679289557
ITERATION #9
	Updates: time=0.06
	Rate:0.023875159262233134
ITERATION #10
	Updates: time=0.14
	Rate:0.0202149065816812
ITERATION #11
	Updates: time=0.12
	Rate:0.01649233657459847
ITERATION #12
	Updates: time=0.10
	Rate:0.013244539975868965
ITERATION #13
	Updates: time=0.09
	Rate:0.011042456587877057
ITERATION #14
	Updates: time=0.12
	Rate:0.009528857601466701
ITERATION #15
	Updates: time=0.14
	Rate:0.008225399235167137
ITERATION #16
	Updates: time=0.15
	Rate:0.0068061441956668795

KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
@author: ogouvert
"""

import os
import glob
import pandas as pd
import pickle as pickle
import numpy as np
import sys

sys.path.append("../../model/OrdNMF")
sys.path.append("../../model/dcPF")
sys.path.append("../../function")

from scipy.stats import poisson
import preprocess_data  as prep
import rec_eval

#%% Pre-processed data 
with open('../../data/ML/ml_145_U2.00e+04_I1.19e+04_min_uc20_sc20', 'rb') as f:
    out = pickle.load(f)
    Y = out['Y_listen']
U,I = Y.shape
T = Y.max()

#%%
prop_test = 0.2
Seed_train_test = [1001] # seed

for seed_train_test in Seed_train_test:
    Y_train,Y_test = prep.divide_train_test(Y,prop_test=prop_test,seed=seed_train_test)
    u,i = Y_test.nonzero()
    yt = (Y_test.data[:,np.newaxis] == np.arange(T+1)[np.newaxis,:])

    save_dir = 'out/seed_%d' %(seed_train_test)

    for filename in glob.glob(os.path.join(save_dir,'*')):  
        print filename
        save=False
        with open(filename,'rb') as f:
            model = pickle.load(f)
            W = model.Ew
            H = model.Eh
        #model.score={} - erase the score
        for s in range(10):
            if ~np.isin('ndcg@100s'+str(s), model.score.keys()):
                save=True
                ndcg = rec_eval.normalized_dcg_at_k(Y_train>0,Y_test>s,W,H,k=100)
                model.score['ndcg@100s'+str(s)]=ndcg
        if save == True:
            model.save_dir = save_dir
            model.save_model()

#%% Read scores
appended_data =[]
for seed_train_test in Seed_train_test:
    save_dir = 'out/seed_%d' %(seed_train_test)
    for filename in glob.glob(os.path.join(save_dir,'*')):  
        with open(filename,'rb') as f:
            model = pickle.load(f)
        df_name = pd.DataFrame.from_dict([{'filename':filename, 
                                           'classname':model.classname}])
        df_init = pd.DataFrame.from_dict([model.saved_args_init])
        df_fit = pd.DataFrame.from_dict([model.saved_args_fit])
        df_score = pd.DataFrame.from_dict([model.score])
        df_loc = pd.concat([df_name,df_init,df_fit,df_score], axis=1)
        if model.classname =='dcpf_Log':
            df_loc['en moyen'] = model.En.data.mean()
            df_loc['T'] = 0
            df_loc['approx'] = False
        else:
            df_loc['en moyen'] = model.En.mean()
        appended_data.append(df_loc)
      
if appended_data!=[]:
    df = pd.concat(appended_data, axis=0)

#%%
columns = [u'classname', u'filename', u'prefix',
           u'K', u'T', u'approx', 
           u'en moyen',
           u'ndcg@100s0', u'ndcg@100s3',u'ndcg@100s5',
           u'ndcg@100s7', u'ndcg@100s9',
           u'loglik']
df = df[columns] 

res = df.groupby(['classname','prefix','T','approx','K']).mean().reset_index()

#%%
sc = 'ndcg@100s7'
idx = res.groupby(['classname','prefix','T','approx'])[sc].transform(max) == res[sc]
res = res[idx]

In [4]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
@author: ogouvert
"""

import os
import pickle as pickle
import scipy.sparse as sparse
import numpy as np
import sys

sys.path.append("../../model/OrdNMF")
sys.path.append("../../function")

from OrdNMF import Ord_generate

import preprocess_data  as prep

import matplotlib.pyplot as plt

#%%
prop_test = 0.2
seed_test = 1001

with open('../../data/ML/ml_145_U1.29e+02_I4.72e+02_min_uc20_sc20', 'rb') as f:
    out = pickle.load(f)
    Y = out['Y_listen']
U,I = Y.shape
T = Y.max()

Y_train,Y_test = prep.divide_train_test(Y,prop_test=prop_test,seed=seed_test)
save_dir = 'out/seed_%d' %(seed_test)
y = Y_train.data
D = float(U*I)

####################
info = np.unique(y,return_counts=True)
sparsity_truth = Y_train.nnz/D*100.
index_truth = info[0]
h_truth = info[1]

####################
#%% OrdNMF
filename = 'ML_OrdNMF_K150_T10_alpha0.30_0.30_beta1.00_1.00_opthyper_beta_approxN_False_tol1.0e-05_seed0'
with open(os.path.join(save_dir,filename),'rb') as f:
    model = pickle.load(f)
    
W = model.Ew
H = model.Eh
L = W.dot(H.T)

theta = model.theta

#%%
np.random.seed(0)

Yppc = Ord_generate(L,theta[:-1])
Yppc = sparse.csr_matrix(Yppc)
yppc = Yppc.data

#%%
info = np.unique(yppc,return_counts=True)
index_onmf = info[0]
h_ppc_onmf = info[1]
sparsity_onmf = Yppc.nnz/D*100.

####################
#%% PLOT 
n_groups = 10

# create plot
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.set_ylim([10**4, 10**7])

index = np.arange(n_groups)
bar_width = 0.3
opacity = .5

rects1 = ax.bar(index_truth, h_truth, bar_width,
                bottom=1,
                alpha=opacity,color='b',
                label=r"Truth ($%.2f$)"%sparsity_truth)
rects2 = ax.bar(index_onmf + bar_width, h_ppc_onmf, bar_width,
                bottom=1,
                alpha=opacity,color='r',
                label=r"OrdNMF ($%.2f$)"%sparsity_onmf)

plt.xlabel('Class')
plt.ylabel("Occurence number")
plt.xticks(index+1.1 ,np.arange(n_groups)+1)
plt.legend(loc=1)

plt.tight_layout()

plt.savefig('fig/PPC_ML.pdf',format='pdf', dpi=1200)

FileNotFoundError: [Errno 2] No such file or directory: 'out/seed_1001/ML_OrdNMF_K150_T10_alpha0.30_0.30_beta1.00_1.00_opthyper_beta_approxN_False_tol1.0e-05_seed0'